In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [3]:
df = pd.read_csv('streamer_df')

In [33]:
def get_data(url, pages):
    import time
    t=5
    df = parse_html(f'{url}=1') # getting the first page
    pages = range(2,pages)
    for page in pages: # getting all other pages, defined by argument
        print(page)
        new_df = parse_html(f"{url}{page}")
        df = df.append(new_df, ignore_index=True)
        time.sleep(t)
    return df

def parse_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    df=pd.read_html(str(soup), flavor='bs4')[0]
    return df

# creates a dataframe for social media plattforms scraped from twitch for a given df with usernames in it
def get_social(df):
    ''' this function scrapes the URL to youtube and twitter for a username-list'''
    import time
    t=5
    usernames = df.username.to_list()
    options = webdriver.ChromeOptions()
    options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
    chrome_driver_binary = "/Users/frckles/drivers/chromedriver"
    dict = {}
    for user in usernames:
        URL= f'https://www.twitch.tv/{user}/about'
        driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
        driver.get(URL)
        soup=BeautifulSoup(driver.page_source)
        twitter = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "twitter.com/" in a["href"]]))
        youtube = list(set([a["href"] for a in soup.find_all("a", href=True) if "youtube.com/" in a["href"]]))
        instagram = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "instagram" in a["href"]]))
        discord = list(set([a["href"].lower() for a in soup.find_all("a", href=True) if "discord" in a["href"]]))
        if twitter:
            twitter = twitter[0]
        if instagram:
            instagram = instagram[0]
        if discord:
            discord = discord[0]
        dict[user] = [twitter, youtube, instagram, discord]
        time.sleep(t)
    df = pd.DataFrame.from_dict(dict, orient='index')
    df['Twitter'] = df[0].apply(lambda x: None if x == [] else x)
    df['YouTube'] = df[1].apply(lambda x: None if x == [] else x)
    df['Instagram'] = df[2].apply(lambda x: None if x == [] else x)
    df['Discord'] = df[3].apply(lambda x: None if x == [] else x)
    df.index.name = 'username'
    df = df.reset_index()
    return df[['username', 'Twitter', 'YouTube', 'Instagram', 'Discord']]

# transfer stuff to csv
# df.to_csv(r'/Users/home/code/Awesimous/StarFish/notebooks/put_name', index=False)

In [36]:
socials = get_social(df.head(10))

<ipython-input-33-559cea6ad27e>:32: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)


In [37]:
df_socs = socials.copy()
df_socs

,username,Twitter,YouTube,Instagram,Discord
0,tommyinnit,https://twitter.com/tommyinnit,None,https://www.instagram.com/tommyinnitt/,https://discord.gg/an6rt4d
1,Quackity,https://twitter.com/quackity,"[https://www.youtube.com/user/QuackityHQ, http...",https://www.instagram.com/quackity/?hl=en,https://www.discord.gg/quackity
2,Sapnap,https://twitter.com/sapnap,"[https://www.youtube.com/sapnap, https://www.y...",https://www.instagram.com/sapnapinsta/,https://discord.gg/qwmnrnp
3,auronplay,https://twitter.com/auronplay,[https://www.youtube.com/channel/UCFR2oaNj02Wn...,https://www.instagram.com/auronplay/,None
4,xQcOW,https://twitter.com/xqc,[https://www.youtube.com/channel/UCmDTrq0LNgPo...,None,https://discord.gg/xqcow
5,AdinRoss,https://twitter.com/adinross,[https://www.youtube.com/channel/UCpq0Ou2ObQr-...,https://www.instagram.com/adin.ross,None
6,WilburSoot,None,None,None,None
7,ibai,https://twitter.com/ibaillanos,[https://www.youtube.com/channel/UCaY_-ksFSQtT...,https://www.instagram.com/ibaillanos/,None
8,Tubbo,https://twitter.com/tubbolive,"[https://www.youtube.com/tubbo, https://www.yo...",None,https://discord.gg/tubbo
9,LCK_Korea,None,[https://www.youtube.com/channel/UCw1DsweY9b2A...,https://www.instagram.com/lckofficial/?hl=ko,None


In [40]:
df_socs['YouTube'][1]

['https://www.youtube.com/user/QuackityHQ',
 'http://youtube.com/c/Quackitwo',
 'https://www.youtube.com/channel/UCFbZ2e9IrPejOdp8wsKUxvA/',
 'https://www.youtube.com/c/Quackity']

In [61]:
test_link =  ['https://www.youtube.com/channel/UCFbZ2e9IrPejOdp8wsKUxvA?view=srkgjsef&loc=xmisland','https://www.youtube.com/channel/UCFbZ2e9IrPejOdp8IM-FAKE?view=srkgjsef&loc=xmisland',  'https://www.youtube.com/channel/short_boi?view=srkgjsef&loc=xmisland']

In [86]:
def get_social_id(link):
    '''Extract streamer identifier at the end of their social media links'''
    if link[-1] == '/':
        link = link[:-1]
    start = link.rfind('/')
    id = link[start + 1:]

    return id

def get_yt_channel_id(links):
    '''Return the first genuine yt channel id in a list of yt links'''
    if not links:
        return None
    channel_id = None
    for link in links:
        link = link.split('?')[0]
        print(link)
        id = get_social_id(link)
        if len(id) == 24:
            channel_id = id
            break
    
    return channel_id


def parse_streamer_socials(socials_df):
    '''Return a dataframe with relevant ids per treamer and platform'''
    for (column_name, column_data) in socials_df.iteritems():
        if column_name != 'Discord':
            if column_name == 'YouTube':
                socials_df[column_name] = socials_df[column_name].apply(get_yt_channel_id)
            socials_df[column_name] = column_data.apply(lambda x: get_social_id(x) if x else None)
            
        
    return socials_df

In [87]:
get_yt_channel_id(test_link)

https://www.youtube.com/channel/UCFbZ2e9IrPejOdp8wsKUxvA


'UCFbZ2e9IrPejOdp8wsKUxvA'

In [88]:
parse_streamer_socials(df_socs)

https://www.youtube.com/user/QuackityHQ
http://youtube.com/c/Quackitwo
https://www.youtube.com/channel/UCFbZ2e9IrPejOdp8wsKUxvA/
https://www.youtube.com/sapnap
https://www.youtube.com/channel/UCEKH2P285Qylr60_e32Xn_g
https://www.youtube.com/channel/UCFR2oaNj02WnXkOgLH0iqOA
https://www.youtube.com/channel/UCmDTrq0LNgPodDOFZiSbsww
https://www.youtube.com/channel/UCpq0Ou2ObQr-LbTB6hsl8BA
https://www.youtube.com/channel/UCaY_-ksFSQtTGk0y1HA_3YQ
https://www.youtube.com/tubbo
https://www.youtube.com/channel/UCAz5JW1_oryewk0eR-eP7Bw
https://www.youtube.com/channel/UCw1DsweY9b2AKGjV4kGJP1A


,username,Twitter,YouTube,Instagram,Discord
0,tommyinnit,tommyinnit,None,tommyinnitt,https://discord.gg/an6rt4d
1,Quackity,quackity,UCFbZ2e9IrPejOdp8wsKUxvA,?hl=en,https://www.discord.gg/quackity
2,Sapnap,sapnap,UCEKH2P285Qylr60_e32Xn_g,sapnapinsta,https://discord.gg/qwmnrnp
3,auronplay,auronplay,UCFR2oaNj02WnXkOgLH0iqOA,auronplay,None
4,xQcOW,xqc,UCmDTrq0LNgPodDOFZiSbsww,None,https://discord.gg/xqcow
5,AdinRoss,adinross,UCpq0Ou2ObQr-LbTB6hsl8BA,adin.ross,None
6,WilburSoot,None,None,None,None
7,ibai,ibaillanos,UCaY_-ksFSQtTGk0y1HA_3YQ,ibaillanos,None
8,Tubbo,tubbolive,UCAz5JW1_oryewk0eR-eP7Bw,None,https://discord.gg/tubbo
9,LCK_Korea,None,UCw1DsweY9b2AKGjV4kGJP1A,?hl=ko,None


TODO: Insta ids need attention!!